<a href="https://colab.research.google.com/github/jh941213/kaggle_NFL/blob/main/nfl_competition_xgboost_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

XGBoost 란?
- XGBoost는 Extreme Gradient Boosting의 약자이다.
- Boosting 기법을 이용하여 구현한 알고리즘은 Gradient Boost 가 대표적인데
이 알고리즘을 병렬 학습이 지원되도록 구현한 라이브러리가 XGBoost 이다.
- Regression, Classification 문제를 모두 지원하며, 성능과 자원 효율이 좋아서, 인기 있게 사용되는 알고리즘이다.

XGBoost의 장점
- GBM 대비 빠른 수행시간
- 병렬 처리로 학습, 분류 속도가 빠르다.
- 과적합 규제(Regularization)
- 표준 GBM 경우 과적합 규제기능이 없으나, XGBoost는 자체에 과적합 규제 기능으로 강한 내구성 지닌다.
- 분류와 회귀영역에서 뛰어난 예측 성능 발휘
- 즉, CART(Classification and regression tree) 앙상블 모델을 사용
- Early Stopping(조기 종료) 기능이 있음
- 다양한 옵션을 제공하며 Customizing이 용이하다.


참조 : https://wooono.tistory.com/97

In [ ]:
!pip install kaggle --upgrade # kaggle install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/kaggle.json #kaggle json API

In [ ]:
!kaggle competitions download -c nfl-player-contact-detection #kaggle NFL data download


100% 3.83G/3.84G [02:57<00:00, 23.1MB/s]
100% 3.84G/3.84G [02:57<00:00, 23.1MB/s]


In [ ]:
import zipfile

z= zipfile.ZipFile('titanic.zip')
z.extractall() #unzip

In [ ]:
import os
import torch
#환경설정 변수 
class Config:
    AUTHOR = "colum2131" #작성자 이름 

    NAME = "NFLC-" + "Exp001-simple-xgb-baseline" # 프로젝트 네임

    COMPETITION = "nfl-player-contact-detection" #대회 이름

    seed = 42 # 랜덤시드 변수
    num_fold = 5 # 교차검증에 사용되는 fold의 수다
    
    xgb_params = {  # XGBoost model 의 하이퍼 파라미터 
        'objective': 'binary:logistic', # 손실함수를 나타낸다 : 이진분류의 경우 로지스틱
        'eval_metric': 'auc', # 평가지표를 acurracy
        'learning_rate':0.03, # 학습률을 나타내느 부동 소수점 변수 0.03 보통 우리가 배웠을땐 0.005 로 하긴했었다 수정해볼 필요 있어보인다
        'tree_method':'hist' if not torch.cuda.is_available() else 'gpu_hist'
    } # 트리구성 알고리즘을 나타내는 변수 : gpu 가 사용불가할때 gpu_hist 로 설정이 된다.

In [ ]:
!pip install cudf-cu11==22.12 rmm-cu11==22.12 --extra-index-url=https://pypi.ngc.nvidia.com/. #cudf 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com/.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.6/453.6 KB 51.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follo

In [ ]:
!git clone https://github.com/rapidsai/cuml.git # cuml git clone

Cloning into 'cuml'...
remote: Enumerating objects: 126864, done.
remote: Counting objects: 100% (832/832), done.
remote: Compressing objects: 100% (495/495), done.
remote: Total 126864 (delta 342), reused 768 (delta 321), pack-reused 126032
Receiving objects: 100% (126864/126864), 153.93 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (97842/97842), done.


In [ ]:
!pip install cuml # cuml install 인데 cuda 버전과 상이해서 오류가나는 것으로 생각된다

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cuml-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cuml
  Running setup.py clean for cuml
Failed to build cuml
  error: subprocess-exited-with-error
  
  × Running setup.py install for cuml did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for cuml ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cuml

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the f

In [ ]:
!pip install cudf
!pip install cupy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cudf
  Running setup.py clean for cudf
Failed to build cudf
  error: subprocess-exited-with-error
  
  × Running setup.py install for cudf did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for cudf ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cudf

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
Looking in indexes: https://pypi.org/sim

In [ ]:
import os
import gc
import subprocess # 하위 프로세서를 다루는 모듈

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import Video, display

from scipy.optimize import minimize #과학적인 계산을 위한 모듈
import cv2
from glob import glob
from tqdm import tqdm

from sklearn.model_selection import GroupKFold #모델선택을 위한 모듈 Group Kfold
from sklearn.metrics import ( # 평가지표를 위한 모델
    roc_auc_score,
    matthews_corrcoef,
)

import xgboost as xgb

import torch

if torch.cuda.is_available():
    import cupy #numpy 와 유사한 인터페이스를 가지는 Gpu용 배열 라이브러리
    import cudf # gpu를 사용하여 데이터 프레임을 조작하는 라이브러리 
    from cuml import ForestInference # gpu를 사용하여 머신러닝 알고리즘을 실행하는 라이브러리

ModuleNotFoundError: ignored

In [ ]:
pip install cuml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cuml-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cuml
  Running setup.py clean for cuml
Failed to build cuml
  error: subprocess-exited-with-error
  
  × Running setup.py install for cuml did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for cuml ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cuml

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the f

In [ ]:
def setup(cfg): # 환경설정 함수
    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #현재 사용가능한 디바이스를 확인하고 gpu 가 사용 가능하다면 쿠다를 아니면 CPU
    
    # set dirs
    cfg.INPUT = f'../input/{cfg.COMPETITION}'   #AUTHOR = "colum2131" #작성자 이름  NAME = "NFLC-" + "Exp001-simple-xgb-baseline" # 프로젝트 네임 #COMPETITION = "nfl-player-contact-detection" #대회 이름
    cfg.EXP = cfg.NAME  #"NFLC-" + "Exp001-simple-xgb-baseline"
    cfg.OUTPUT_EXP = cfg.NAME #"NFLC-" + "Exp001-simple-xgb-baseline"
    cfg.SUBMISSION = './'
    cfg.DATASET = '../input/'

    cfg.EXP_MODEL = os.path.join(cfg.EXP, 'model') #NFLC-" + "Exp001-simple-xgb-baseline/model
    cfg.EXP_FIG = os.path.join(cfg.EXP, 'fig')     #NFLC-" + "Exp001-simple-xgb-baseline/fig
    cfg.EXP_PREDS = os.path.join(cfg.EXP, 'preds') #NFLC-" + "Exp001-simple-xgb-baseline/preds

    # make dirs
    for d in [cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
        os.makedirs(d, exist_ok=True)
        
    return cfg

In [ ]:
# ==============================
# function
# ==============================
# ref: https://www.kaggle.com/code/robikscube/nfl-player-contact-detection-getting-started


def add_contact_id(df): # df game_play, step, nfl_player_id_1, nfl_player_id_2 열을 조합하여 contact_id 를 만들어낸다
    # Create contact ids  
    df["contact_id"] = (df["game_play"]+ "_" + df["step"].astype("str")+ "_" + df["nfl_player_id_1"].astype("str") + "_" + df["nfl_player_id_2"].astype("str"))
    return df

def expand_contact_id(df):
    """
    contact_id를 별도의 열로 분할합니다.
    """
    df["game_play"] = df["contact_id"].str[:12] # contact_id 에서 열에서 12자리를 가져온다
    df["step"] = df["contact_id"].str.split("_").str[-3].astype("int") # 열을 _로 분리한후 뒤에서 3번째 정수형으로 변환하여 가져온다
    df["nfl_player_id_1"] = df["contact_id"].str.split("_").str[-2] # 열을 _로 분리후 뒤에서 2번째 문자열로 가져온다 플레이어 ID1
    df["nfl_player_id_2"] = df["contact_id"].str.split("_").str[-1] # 열을 _로 분리후 뒤에서 1번째 문자열로 가져온다 플레이어 ID2
    return df #새로만든 df 데이터 프레임을 리턴해준다


# cross validation 
                   #(train 'contact', 'game_play', cfg.num_fold)
def get_groupkfold(train, target_col, group_col, n_splits): # train 데이터와 target_col, group_col, n_splits를 입력값으로 받습니다.
    kf = GroupKFold(n_splits=n_splits) #GroupKFold 객체를 생성하여 train 데이터를 group_col로 그룹화하고 n_splits 수 만큼 fold를 나누는 교차 검증을 수행합니다.
    generator = kf.split(train, train[target_col], train[group_col]) #target_col 타겟변수의 열이 름 지정 , group_col 그룹별 교차검증 위해 사용
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator): # fold별로 validation set의 인덱스를 저장하여 fold_series 리스트에 추가합니다.
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index() # 마지막으로 fold_series를 concatenate하고 인덱스를 정렬하여 반환합니다.
    return fold_series 

# xgboost code
#fit_xgboost(cfg, train_X, train_y, cfg.xgb_params, add_suffix="_xgb_1st")
def fit_xgboost(cfg, X, y, params, add_suffix=''): #환경변수 , Train X ,Train y , params , 모델 이름에 추가할 접미사?
    """
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate':0.01,
        'tree_method':'gpu_hist'
    }
    """
   
    oof_pred = np.zeros(len(y), dtype=np.float32)       #배열은 XGBoost 모델의 훈련 중 생성된 out-of-fold 예측값을 저장하는 데 사용됩니다.
    for fold in sorted(cfg.folds.unique()): # 1. 현재 fold에 해당하는 인덱스로 학습 데이터와 검증 데이터를 나눕니다.
        if fold == -1: continue
        idx_train = (cfg.folds!=fold)
        idx_valid = (cfg.folds==fold)
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        display(pd.Series(y_valid).value_counts())

        xgb_train = xgb.DMatrix(x_train, label=y_train) #2. 나눠진 학습/검증 데이터를 xgboost에서 사용할 수 있는 형태로 변환합니다.
        xgb_valid = xgb.DMatrix(x_valid, label=y_valid)
        evals = [(xgb_train,'train'),(xgb_valid,'eval')]

        model = xgb.train( # 3. xgboost 모델을 학습합니다.
            params,
            xgb_train,
            num_boost_round=10_000, #100번중에 계산이 안됨
            early_stopping_rounds=100,
            evals=evals,
            verbose_eval=100,
        )

        model_path = os.path.join(cfg.EXP_MODEL, f'xgb_fold{fold}{add_suffix}.model') # 모델이 저장될 경로
        model.save_model(model_path)#4. 학습된 모델을 디스크에 저장합니다.

        if not torch.cuda.is_available(): #
            model = xgb.Booster().load_model(model_path)   #xgb.Booster().load_model() 메서드를 사용하여 지정된 경로에서 훈련된 XGBoost 모델을 로드하고 
        else:
            model = ForestInference.load(model_path, output_class=True, model_type='xgboost') #그렇지 않으면 ForestInference.load() 메서드를 사용하여 지정된 매개변수로 모델을 로드합니다.
        pred_i = model.predict_proba(x_valid)[:, 1]        #그런 다음 모델.predict_proba() 메서드를 사용하여 검증 세트 x_valid에서 예측을 수행하고 [:, 1]로 
        oof_pred[x_valid.index] = pred_i                   #결과를 슬라이스하여 클래스 1의 확률을 추출합니다. 예측된 확률은 x_valid.index를 사용하여 oof_pred의 해당 인덱스에 할당됩니다.
        score = round(roc_auc_score(y_valid, pred_i), 5) 
        print(f'Performance of the prediction: {score}\n') #모델의 성능은 roc_auc_score() 함수를 사용하여 true 라벨인 y_valid와 예측된 확률 pred_i로 평가되며 결과는 소수점 5 자리로 표시됩니다.
        del model; gc.collect()                            #마지막으로, 로드된 모델은 del model 및 gc.collect()를 사용하여 삭제되고 메모리가 해제됩니다.

    np.save(os.path.join(cfg.EXP_PREDS, f'oof_pred{add_suffix}'), oof_pred) #np.save를 사용하여 배열을 지정된경로에 저장하고 acc함수를 사용하여 레이블과 y와 예측간 AUC 점수를 계산
    score = round(roc_auc_score(y, oof_pred), 5) #계산된 점수는 5자리로 반올림하여 콘솔에 출력됩니다.
    print(f'All Performance of the prediction: {score}') 
    return oof_pred #oof_pred 배열 반환해준다


# pred_xgboost(test_X, cfg.EXP_MODEL, add_suffix="_xgb_1st") 
def pred_xgboost(X, data_dir, add_suffix=''):
    models = glob(os.path.join(data_dir, f'xgb_fold*{add_suffix}.model'))   #이 함수는 입력 특징 X와 디렉토리 경로 data_dir를 입력으로 받고, 디렉토리에 저장된 XGBoost 모델의 예측값을 반환합니다. add_suffix 인수는 파일 이름에 특정 접미사가 있는 모델을 선택할 수 있도록 합니다.
    if not torch.cuda.is_available():
         models = [xgb.Booster().load_model(model_path) for model in models]
    else:
        models = [ForestInference.load(model, output_class=True, model_type='xgboost') for model in models]
    preds = np.array([model.predict_proba(X)[:, 1] for model in models])
    preds = np.mean(preds, axis=0)
    return preds #이 함수는 XGBoost 모델이 이진 결과를 예측하도록 학습되었다고 가정합니다 (예: 0 또는 1). 이 함수는 확률을 추출하기 위해 [:, 1] 인덱싱을 사용하기 때문에, 모델이 다중 클래스를 예측하도록 학습된 경우 예측을 올바르게 처리하기 위해 수정해야 할 수도 있습니다.

In [ ]:
# ==============================
# read data
# ==============================
cfg = setup(Config)

if not torch.cuda.is_available():
    tr_tracking = pd.read_csv(os.path.join(cfg.INPUT, 'train_player_tracking.csv'), parse_dates=["datetime"])
    te_tracking = pd.read_csv(os.path.join(cfg.INPUT, 'test_player_tracking.csv'), parse_dates=["datetime"])
    # tr_helmets = pd.read_csv(os.path.join(cfg.INPUT, 'train_baseline_helmets.csv'))
    # te_helmets = pd.read_csv(os.path.join(cfg.INPUT, 'test_baseline_helmets.csv'))
    # tr_video_metadata = pd.read_csv(os.path.join(cfg.INPUT, 'train_video_metadata.csv'))
    # te_video_metadata = pd.read_csv(os.path.join(cfg.INPUT, 'test_video_metadata.csv'))
    sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

    train = pd.read_csv(os.path.join(cfg.INPUT, 'train_labels.csv'), parse_dates=["datetime"])
    test = expand_contact_id(sub)
    
else:
    tr_tracking = cudf.read_csv(os.path.join(cfg.INPUT, 'train_player_tracking.csv'), parse_dates=["datetime"])
    te_tracking = cudf.read_csv(os.path.join(cfg.INPUT, 'test_player_tracking.csv'), parse_dates=["datetime"])
    # tr_helmets = cudf.read_csv(os.path.join(cfg.INPUT, 'train_baseline_helmets.csv'))
    # te_helmets = cudf.read_csv(os.path.join(cfg.INPUT, 'test_baseline_helmets.csv'))
    # tr_video_metadata = cudf.read_csv(os.path.join(cfg.INPUT, 'train_video_metadata.csv'))
    # te_video_metadata = cudf.read_csv(os.path.join(cfg.INPUT, 'test_video_metadata.csv'))
    sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

    train = cudf.read_csv(os.path.join(cfg.INPUT, 'train_labels.csv'), parse_dates=["datetime"])
    test = cudf.DataFrame(expand_contact_id(sub))

FileNotFoundError: ignored

The following code is used to create the features.  
Basically, the numerical features contained in player_tracking.csv are merged into player_id_1 and player_id_2 respectively.

In [ ]:
# ==============================
# feature engineering
# ==============================

# 데이터셋 목적에 맞게 변환해주는 작업같음

def create_features(df, tr_tracking, merge_col="step", use_cols=["x_position", "y_position"]):
    output_cols = []
    df_combo = (
        df.astype({"nfl_player_id_1": "str"}) #type 변경 nf1_player_Id_1 : str
        .merge( tr_tracking.astype({"nfl_player_id": "str"})[["game_play", merge_col, "nfl_player_id",] + use_cols],
            left_on=["game_play", merge_col, "nfl_player_id_1"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        ) # 
        .rename(columns={c: c+"_1" for c in use_cols})
        .drop("nfl_player_id", axis=1)
        .merge(
            tr_tracking.astype({"nfl_player_id": "str"})[
                ["game_play", merge_col, "nfl_player_id"] + use_cols
            ],
            left_on=["game_play", merge_col, "nfl_player_id_2"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        )
        .drop("nfl_player_id", axis=1)
        .rename(columns={c: c+"_2" for c in use_cols})
        .sort_values(["game_play", merge_col, "nfl_player_id_1", "nfl_player_id_2"])
        .reset_index(drop=True)
    )
    output_cols += [c+"_1" for c in use_cols]
    output_cols += [c+"_2" for c in use_cols]
    
    if ("x_position" in use_cols) & ("y_position" in use_cols):
        index = df_combo['x_position_2'].notnull()
        if torch.cuda.is_available():
            index = index.to_array()
        distance_arr = np.full(len(index), np.nan)
        tmp_distance_arr = np.sqrt(
            np.square(df_combo.loc[index, "x_position_1"] - df_combo.loc[index, "x_position_2"])
            + np.square(df_combo.loc[index, "y_position_1"]- df_combo.loc[index, "y_position_2"])
        )
        if torch.cuda.is_available():
            tmp_distance_arr = tmp_distance_arr.to_array()
        distance_arr[index] = tmp_distance_arr
        df_combo['distance'] = distance_arr
        output_cols += ["distance"]
        
    df_combo['G_flug'] = (df_combo['nfl_player_id_2']=="G")
    output_cols += ["G_flug"]
    return df_combo, output_cols


use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa'
]
train, feature_cols = create_features(train, tr_tracking, use_cols=use_cols)
test, feature_cols = create_features(test, te_tracking, use_cols=use_cols)
if torch.cuda.is_available():
    train = train.to_pandas()
    test = test.to_pandas()

display(train)
'''
주어진 데이터프레임(df)과 위치 추적 데이터프레임(tr_tracking)을 조합하여 새로운 특징(feature)들을 만드는 함수 create_features를 정의합니다. 
create_features 함수는 데이터프레임(df_combo)과 출력 열 목록(output_cols)을 반환합니다.

먼저, df_combo는 다음 단계(step) 또는 병합 열(merge_col)과 nfl_player_id_1 열을 기준으로 데이터프레임(df)과 위치 추적 데이터프레임(tr_tracking)을 병합합니다. 
이후, use_cols에서 지정한 열에 대해 "_1"을 추가한 이름으로 열 이름을 바꾸고, nfl_player_id 열을 삭제합니다.

그 다음, nfl_player_id_2 열을 기준으로 데이터프레임(df_combo)과 위치 추적 데이터프레임(tr_tracking)을 병합합니다. 이후, use_cols에서 지정한 열에 대해 "_2"를 추가한 이름으로 열 이름을 바꾸고, 
nfl_player_id 열을 삭제합니다.

그리고 마지막으로, game_play, merge_col, nfl_player_id_1 및 nfl_player_id_2 열을 기준으로 데이터프레임(df_combo)을 정렬합니다.
 만약 use_cols에 "x_position"과 "y_position"이 포함되어 있다면, 데이터프레임(df_combo)에 "distance" 열을 추가하고, nfl_player_id_2가 "G"인 경우 "G_flug" 열을 추가합니다.

마지막으로, create_features 함수는 데이터프레임(df_combo)과 출력 열 목록(output_cols)을 반환하고, 
use_cols에서 지정한 열을 사용하여 훈련 데이터(train)와 테스트 데이터(test)에 대해 create_features 함수를 호출합니다. 마지막으로, train 데이터프레임을 출력합니다.

'''

NameError: ignored

In [ ]:
# ==============================
# training & inference
# ==============================
train_X = train[feature_cols] #create_features 함수를 사용하여 피쳐 컬럼을 선택하고, train_X와 test_X에 할당합니다.
test_X = test[feature_cols]
train_y = train['contact'] #타겟 변수 contact를 train_y에 할당합니다.
cfg.folds = get_groupkfold(train, 'contact', 'game_play', cfg.num_fold) #get_groupkfold 함수를 사용하여 train 데이터셋에서 game_play을 그룹화 변수로 사용하여 그룹 k-폴드 교차 검증을 수행합니다. 결과적인 폴드는 CSV 파일로 저장됩니다.
cfg.folds.to_csv(os.path.join(cfg.EXP_PREDS, 'folds.csv'), index=False)#결과적인 폴드는 CSV 파일로 저장됩니다.

oof_pred = fit_xgboost(cfg, train_X, train_y, cfg.xgb_params, add_suffix="_xgb_1st") #fit_xgboost 함수를 사용하여 지정된 하이퍼파라미터 cfg.xgb_params를 사용하여 train_X와 train_y 데이터를 기반으로 XGBoost 모델을 학습합니다. 결과적으로 나온 교차 검증 예측은 oof_pred에 할당됩니다.
sub_pred = pred_xgboost(test_X, cfg.EXP_MODEL, add_suffix="_xgb_1st") #pred_xgboost 함수를 사용하여 학습된 XGBoost 모델을 사용하여 test_X 데이터에 대한 예측을 생성합니다. 결과적인 예측은 sub_pred에 할당됩니다.

NameError: ignored

In [ ]:
# ==============================
# optimize
# ==============================
#목적 함수정의
def func(x_list):
    score = matthews_corrcoef(train['contact'], oof_pred>x_list[0])
    return -score
#초기값 설정 및 최적화 수행
x0 = [0.5]
result = minimize(func, x0,  method="nelder-mead")
#초기값 설정 및 최적화 수행
cfg.threshold = result.x[0]
print("score:", round(matthews_corrcoef(train['contact'], oof_pred>cfg.threshold), 5))
print("threshold", round(cfg.threshold, 5))
#결과 예측
test = add_contact_id(test)
test['contact'] = (sub_pred > cfg.threshold).astype(int)
test[['contact_id', 'contact']].to_csv('submission.csv', index=False)
display(test[['contact_id', 'contact']].head())

NameError: ignored